# 첫 번째 모듈

- 데이터를 학습하고 성능을 평가
- 모델의 성능을 교차 검증을 통해 안정적으로 측정
- 결과를 분석 및 시각화

> 머신러닝 모델 성능 평가

In [1]:
# 모듈이 사용할 패키지
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import GridSearchCV
from pandas import DataFrame

In [2]:
# **kargs: 다양한 하이퍼파라미터(옵션)들을 받을 수 있는 가변 인자
def singleML(modelName, x, y=None, cv=5, **kargs):
    # 모델 생성
    model = modelName(**kargs)
    # 교차 검증
    score = cross_val_score(model, x, y, cv=cv).mean()
    # 결과 DF
    df = DataFrame(cross_validate(model, x, y, cv=cv))
    return [model, score, df]

# 모듈 테스트

In [3]:
from pandas import read_excel
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC

In [4]:
origin = read_excel("https://data.hossam.kr/G02/breast_cancer.xlsx")
x = origin.drop('target', axis=1)
y = origin['target']
x.shape, y.shape

((569, 30), (569,))

In [5]:
# singleML로 선형회귀 수행
model, score, df = singleML(LinearRegression, x, y)
model

LinearRegression()

In [6]:
# 5번의 KFold 평균
score

0.704686173464433

In [7]:
df

,fit_time,score_time,test_score
0,0.010963,0.002027,0.623595
1,0.006992,0.004036,0.698961
2,0.004997,0.002001,0.755933
3,0.008003,0.002025,0.773021
4,0.004998,0.002000,0.671920


In [8]:
model, score, df = singleML(SVC, x, y, kernel='linear', C=0.1, random_state=777)
model

SVC(C=0.1, kernel='linear', random_state=777)

# 두 번째 모듈

GridSearchCV : 머신러닝에서 모델의 성능향상을 위해 쓰이는 기법중 하나. 사용자가 직접 모델의 하이퍼 파라미터의 값을 리스트로 입력하면 값에 대한 경우의 수마다 예측 성능을 측정 평가하여 비교하면서 최적의 하이퍼 파라미터 값을 찾는 과정을 진행.

> 머신러닝 모델 성능 평가

In [14]:
# **kargs: 다양한 하이퍼파라미터(옵션)들을 받을 수 있는 가변 인자
def gridML(modelName, x, y=None, params={}, cv=5, **kargs):
    # 모델 생성
    model = modelName(**kargs)
    # grid 생성
    grid = GridSearchCV(model, param_grid=params, cv=cv)

    try:
        grid.fit(x,y)   # 지도학습
    except:
        grid.fit(x) # 비지도학습

    result_df = DataFrame(grid.cv_results_["params"])
    result_df['mean_test_score'] = grid.cv_results_['mean_test_score']
    result_df.sort_values(by='mean_test_score', ascending=False)
    
    return [grid.best_estimator_, grid.best_score_, result_df]

# 모듈 테스트2

In [15]:
model, best, score = gridML(LinearRegression, x, y)
model

LinearRegression()

In [16]:
score

,mean_test_score
0,0.704686


In [17]:
best

0.704686173464433

In [18]:
params = {
    'C':[0.001, 0.01],
    'kernel':['linear', 'rbf']
}
model, best, score, gridML(SVC, x, y, params=params, random_state=777)
model

LinearRegression()

In [19]:
score

,mean_test_score
0,0.704686


In [20]:
best

0.704686173464433